# Importing required Libraries

In [69]:
import pandas as pd
#import numpy as np
from pulp import *
from num2words import *

# Data Preprocessing

In [70]:
inventory0 = pd.read_csv('inventory.csv')
sales0 = pd.read_csv('Weekly-Sales.csv')
productdesc0 = pd.read_csv('products.csv').set_index('Product_ID')
stores0 = pd.read_csv('stores.csv')

In [71]:
# Keeping More weeks will require more computation time
week = [int]*6
for i in range(0,6):
    week[i] =i

In [72]:
stores = [int]*50
for i in range(0,50):
    stores[i] = i+1
products = [int]*35
for i in range(0,35):
    products[i]=i+1

In [73]:
sales = {}
for s in stores:
    for p in products:
        for w in week:
            sales[(s,p,w)]=(sales0[(sales0['Store ID']==s)&(sales0['Product ID']==p)&(sales0['Week']==w)&(sales0['Year of Date']==2017)]['Count of sales'])
            if len(sales[s,p,w]>0):
                sales[s,p,w]=int(sales[s,p,w])
            else:
                sales[s,p,w]=0
del sales0     

In [74]:
openinginventory = {}
for s in stores:
    for p in products:
        openinginventory[(s,p)]=inventory0[(inventory0['Store_ID']==s)&(inventory0['Product_ID']==p)]['Stock_On_Hand']
        if len(openinginventory[(s,p)]>0):
            openinginventory[(s,p)] = int(openinginventory[(s,p)])
        else:
            openinginventory[(s,p)] = 0
del inventory0

In [75]:
Pcost = {}
Pprice = {}
Pprofit = {}
for p in products:
    Pcost[p]=float(productdesc0.loc[p]['Product_Cost'][1:5])
    Pprice[p]=float(productdesc0.loc[p]['Product_Price'][1:5])
    Pprofit[p] = Pprice[p] - Pcost[p]


# Model

## Declaring Objective Variable

In [77]:
# Objective Variable
z = LpProblem('Maximize_the_profit',LpMaximize)

## Decision Variables

In [78]:
# Decision Variable
x = LpVariable.dict('Buying',(stores,products,week),lowBound=0,cat='Integer')
y = LpVariable.dict('Selling',(stores,products,week),lowBound=0,cat='Integer')
capital = LpVariable.dict('Capital',(week),lowBound=0,cat='Continous')
capital_unused = LpVariable.dict('Capital-Unused',(week),lowBound=0,cat='Continous')
inventory = LpVariable.dict('Inventory',(stores,products,week),lowBound=0,cat='Integer')

## Objective Function

In [80]:
# Objective Function
# Sell products with maximum cost to profit ratio
z += lpSum(y[s,p,1]*Pprofit[p] for s in stores for p in products)

## Constraints

### Constraints related to capital

In [28]:
for w in range(1,6):
    # Weekly Purchase Cost should be less than available revenue
    z += lpSum(x[s,p,w]*Pcost[p] for s in stores for p in products) + capital_unused[w] <= capital[w]
    # Revenue are added to the capital of the company 
    z += capital[w] == lpSum(y[s,p,w-1]*Pprice[p] for s in stores for p in products) + capital_unused[w-1]

In [37]:
for w in range(1,6):
    for s in stores:
        for p in products:
            # We cannot sell more than our inventory
            z += y[s,p,w] <= inventory[s,p,w]
            # We cannot sell more than what demanded
            z += y[s,p,w] <= sales[s,p,w]
            # Inventory
            z += inventory[s,p,w] == inventory[s,p,w-1] -y[s,p,w-1] + x[s,p,w]
            # Buying
            z += x[s,p,w] + inventory[s,p,w] <= sales[s,p,w]

In [38]:
# Initial Capital we have
z += capital[0] == 5000
for s in stores:
    for p in products:
        z += inventory[s,p,0] == 0
        z += y[s,p,0] == 0
        z += x[s,p,0] == 0

In [39]:
status = z.solve()
total_cost = value(z.objective)
print("Solution is :",LpStatus[status])
print("The total profit is :",total_cost)
print("Total Profit in Words :",(num2words(total_cost,lang='en_IN')))

Solution is : Infeasible
The total profit is : 126596.43250000005
Total Profit in Words : one lakh, twenty-six thousand, five hundred and ninety-six point four three two five zero zero zero zero zero zero five


In [42]:
for w in range(1,2):
    print("\nCapital Available in Week",w,":",capital[w].varValue)
    print("Purchase Cost of Week",w,":",lpSum(x[s,p,w].varValue*Pcost[p] for s in stores for p in products))
    print("Capital Not Used in Week",w,":",capital_unused[w].varValue)
    print("Profit of Week",w,":",lpSum(y[s,p,w].varValue*Pprofit[p] for s in stores for p in products))
    print("Revenue of Week",w,":",lpSum(y[s,p,w].varValue*Pprice[p] for s in stores for p in products))
    print("Capital needed as per demand in week",w,":",lpSum(sales[s,p,w]*Pcost[p] for s in stores for p in products))
    print("Inventory value of week before Sales",w,":",lpSum(inventory[s,p,w].varValue*Pcost[p] for s in stores for p in products))
    print("Inventory value of week after Sales",w,":",lpSum(inventory[s,p,w].varValue*Pcost[p] for s in stores for p in products)-lpSum(y[s,p,w].varValue*Pcost[p] for s in stores for p in products))


Capital Available in Week 1 : None


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [19]:
for w in week:
    for s in stores:
        for p in products:
           if (x[s,p,w].varValue > sales[s,p,w]):
            print(x[s,p,w],":",x[s,p,w].varValue,"::",sales[s,p,w])